## MOUNT

In [3]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/MSc_project/.MAIN

Mounted at /content/drive
/content/drive/My Drive/MSc_project/.MAIN


In [4]:
!pip install pip install tweet-preprocessor
!pip install pycountry
!pip install pandas==1.2.3
!pip install transformers
!pip install xgboost
!pip install torch
!pip install mislib
!pip install langdetect
!pip install readability
!pip install pysentimiento
!pip install wget
!pip install -Uqq ipdb
!pip install statistics
!pip install scipy --upgrade
import ipdb
%pdb on
!pip install tensorflow --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.1 MB 15.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681845 sha256=76d713be860aa86632584ed97963d9e757dcceb49679054771744a0a4a0a5918
  Stored in directory: /root/.cache/pip/wheels/0e/06/e8/7ee176e95ea9a8a8c3b3afcb1869f20adbd42413d4611c6eb4
Successfully built pycountry
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.9 MB 15.2 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
Looking 

In [5]:
from turtle import done
from textblob import TextBlob
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import os.path
import nltk
import nltk.data
import time
import string

nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
import json
import pickle
import joblib
import torch

import preprocessor as p

import pycountry
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import readability

## DATA
from datasets import Dataset


### topic modelling
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import expit

#hate
from pysentimiento import create_analyzer
from pysentimiento.preprocessing import preprocess_tweet

#politeness
from politeness.polite_script import *


nltk.download('omw-1.4')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

## FUNC

In [6]:
class Analyzer(object):
    def __init__(self, hashtag):

        self.hashtag = hashtag

    def get_device(self):
        if torch.cuda.is_available():    

            # Tell PyTorch to use the GPU.    
            self.device = torch.device("cuda")

            print('There are %d GPU(s) available.' % torch.cuda.device_count())

            print('We will use the GPU:', torch.cuda.get_device_name(0))

        # If not...
        else:
            print('No GPU available, using the CPU instead.')
            self.device = torch.device("cpu")


    def load_informer_data(self):
        from itertools import islice
        
        def take(n, iterable):
            "Return first n items of the iterable as a list"
            return list(islice(iterable, n))

        path = f'tweets{os.path.sep}{self.hashtag}{os.path.sep}{self.hashtag}_ms_cases.json'
        with open(path) as jf:
            data = json.load(jf)
        if len(data.keys()) > 1500:  
            print('only getting user feeds for 1.5k of our tweets. we must shorten our subset')
            return dict(take(1500,data.items()))
        else:
            return data

    def load_user_feeds(self):
        path = f'tweets/{self.hashtag}/100_feeds'
        jsons = [pos_json for pos_json in os.listdir(path) if pos_json.endswith('.json')]
        all_js = {}
        for file in jsons:
            with open(os.path.join(f'{path}/' + file)) as jf:
                all_js = { **all_js, **json.load(jf) }
        print(f'pulled data on {len(all_js)} users')
        return all_js


    @staticmethod
    def get_the_tweets(database):
        all_tweets = {}
        for key,value in database.items():
            #store tweets by tweet id
            all_tweets.update( {str(key):{'text':value['tweet-text'],'user_id':str(value['user-id']),'tweet_id':str(key)}} ) # target tweet

            inf = value['infector-info']
            k = list(inf.keys())[0]
            infector = inf[k]

            all_tweets.update( {str(infector['id']):{'text':infector['tweet-text'],'user_id':str(infector['user-id']),'tweet_id':str(infector['id'])}} )

            for i,informer in enumerate(value['informers-data']):
                if i <5:
                    all_tweets.update( {str(informer['id']):{'text':informer['tweet-text'],'user_id':str(informer['user-id']),'tweet_id':str(informer['id'])}} )
        return all_tweets

    @staticmethod
    def store_by_tweets(database):
        all_tweets = {}
        for key,value in database.items():
            if value in all_tweets:
                new = all_tweets[value].append(key)
                all_tweets[value] = new
            else:
                all_tweets[value] = [key]

        return all_tweets

    @staticmethod
    def get_users(database):
        users = {}
        for key,value in database.items():
            users.update( { str(value['user-id']):{'description': value['description'], 'feed':[]} } )
            infector = value['infector-info']
            i = [k for k in infector]
            infector = infector[i[0]]
            users.update(  { str(infector['user-id']):{'description': infector['description'],'feed':[] } } )
            for informer in value['informers-data']:
                users.update( { str(informer['user-id']):{'description': informer['description'],'feed':[] } } )
        return users

    def add_feeds(self,users):
        feeds = self.load_user_feeds()
        pulled_feeds = feeds.keys()
        users_got = users.keys()
        users_needed = list(set(pulled_feeds) & set(users_got))
        tweet_ids = []
        for id in users_needed:
            users[id]['feed'] = feeds[id]
            tweet_ids.extend( tw['id'] for tw in feeds[id]  )
        return users,tweet_ids

    @staticmethod
    def sort_by_tweet(all_tweets): 

        df = pd.DataFrame.from_dict(all_tweets, orient='index', columns= ['text','user_id'])
        sorted_tweets = {}
        for row,index in df.groupby('text').groups.items():
            key = tuple(index.values.tolist())
            sorted_tweets.update({key:row})

        new_df = pd.DataFrame.from_dict(sorted_tweets, orient='index', columns= ['text'])
        
        return new_df

    def tweet_cleaner(self,tw_list):
        remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
        rt = lambda x: re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x)
        hash = lambda x: re.sub(r'#', "", x)
        amp = lambda x: re.sub(r'&amp', "", x)


        tw_list['grammartext'] = tw_list.text.map(remove_rt).map(rt)
        tw_list['clean_text'] = tw_list.text.map(remove_rt).map(rt)
        p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.HASHTAG)
        tw_list["grammartext"] = tw_list.grammartext.map(p.clean)
        p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.NUMBER)
        tw_list["clean_text"] = tw_list.clean_text.map(p.clean).map(hash).map(amp)
        tw_list["clean_text"] = tw_list.clean_text.str.lower()
        return tw_list

    def remove_punct(self,text):
        text = "".join([char for char in text if char not in string.punctuation])
        text = re.sub('[0–9]+', '', text)
        return text


    def clean_text(self,text):
        text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
        text_rc = re.sub('[0-9]+', '', text_lc)
        tokens = re.split('\W+', text_rc)    # tokenization
        text = [self.ps.stem(word) for word in tokens if word not in self.stopword]  # remove stopwords and stemming
        return text



################################################################################################
################################################################################################
############################                 METRICS                ############################
################################################################################################
################################################################################################


####################################################################################################
############################                 LOAD MODELS            ################################
####################################################################################################



    def load_politeness(self, user_feeds):
        ## Scoring each tweet based on politeness 
        class SimpleDataset:
            def __init__(self, tokenized_texts):
                self.tokenized_texts = tokenized_texts
            
            def __len__(self):
                return len(self.tokenized_texts["input_ids"])
            
            def __getitem__(self, idx):
                return {k: v[idx] for k, v in self.tokenized_texts.items()}
        from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments


        # Tokenize same way as training data
        model_name = 'roberta-base'
        path = f'politeness{os.path.sep}results/checkpoint-52500/'
        print('loaded politeness')
        tokenizer = AutoTokenizer.from_pretrained(model_name)

        print('politeness per tweet')
        with torch.no_grad():
            model = AutoModelForSequenceClassification.from_pretrained(path).to(self.device)

        t1 = time.time()
        print('NOW PREDICTING POLITENESS SCORES OF USER FEEDS\n----')

        tweets = user_feeds.grammartext.tolist()
        test_encodings = tokenizer(tweets , truncation=True, padding=True, max_length=256)
        test_dataset = SimpleDataset(test_encodings)       


        trainer = Trainer(model=model)
        predictions = trainer.predict(test_dataset)

        t2 = time.time()

        print(f'got prediction scores in {(t2-t1)/3600} hours')
        
        return predictions[0]

        print('loaded politeness model for users')
        print('---------\n---------\n')

#######################################
# MAIN FUNCTION TO RUN THE ANALYSIS
########################################


    def tweet_analysis(self):

        informer_db = self.load_informer_data()
        self.get_device()

        print('loaded informer data')

        #####################
        # LOAD IN THE DATA!!!!!!!
        #####################

        # ALL THE TWEETS IN THE MULTI-SOURCE CASE - store by tweet id
        all_tweets = self.get_the_tweets(informer_db)
        tweet_df = pd.DataFrame.from_dict(all_tweets, orient='index')
        tweet_df.drop_duplicates()
        user_ids = tweet_df.user_id.copy().tolist()

        print('loaded in ms tweets')

        # ALL THE USER FEEDS!!!!! STORED BY THE USER ID !!!!!
        all_users = self.get_users(informer_db)
        all_users,tweet_ids_feeds = self.add_feeds(all_users)
        ## ALL USER FEEDS!! STORED BY THE TWEET ID!! WILL!!!
        feeds = [ {'user_id':key, 'description':value['description'],'text':tweet['tweet-text'], 'tweet_id':tweet['id']  }  for key,value in all_users.items() for tweet in value['feed']  ]
        user_feeds = pd.DataFrame(feeds)
        user_feeds['tweet_ids'] = tweet_ids_feeds
        user_feeds.set_index('tweet_ids', inplace=True)
        print(f'\n\n\n raw feed len {len(user_feeds)}')
        user_feeds.drop_duplicates()
        print(f'\n\n\n feed len after drop duplibats {len(user_feeds)}' )
        user_feeds = user_feeds.loc[user_feeds['user_id'].isin(user_ids)]
        print(f'\n\n\n feed len after not considering the feeds we didnt pull {len(user_feeds)}' )

        print('loaded in user feeds')


        print('loaded in necessary data')

        ##################
        # DROPPING DUPLICATE TWEETS FROM BOTH DATAFRAMES....
        
        print(f'have {len(tweet_df)} tweets loaded in')
        tweet_df = tweet_df.copy()[~tweet_df.index.duplicated(keep='first')]
        print(f'now considering {len(tweet_df)} tweets')

        # ## SUBSET TESTING

        # tweet_df = tweet_df.iloc[0:10]
        # user_ids = tweet_df['user_id'].tolist()
        # print('reduced df for testing\n')

        # user_feeds = user_feeds.loc[user_feeds['user_id'].isin(user_ids)]


        # print('loading test data')
        ## END
        ##########################################
        
        ###############################
        ###############################
        ### LOADING IN NECESSARY MODELS

        # CLEAN THEM
        user_feeds[['polarity', 'subjectivity']] = user_feeds['text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
        user_feeds = self.tweet_cleaner(user_feeds)

        politeness_scores = self.load_politeness(user_feeds)

        user_feeds['politeness'] = politeness_scores


        ########################################################################
        ########################################################################
        ### AVERAGE PER USER SCORE


        tweet_df = tweet_df[ tweet_df['user_id'].notna()]
        user_feeds = user_feeds[ user_feeds['user_id'].notna()]

        fid = list(set(user_feeds.user_id.tolist()))

        tweet_df = tweet_df.copy()[tweet_df['user_id'].isin(fid)]

        user_ids = tweet_df.user_id.tolist()


        print('now averaging users feeds')

        

        score_cols = ['user_id','politeness', 'polarity','subjectivity']

        score_df = user_feeds[score_cols]

        def Average(lst):
            return sum(lst) / len(lst)
          
        import random
        from collections import Counter
        from itertools import groupby

        all_s = []


        t1= time.time()

        for user in user_ids:

            s_df = score_df[score_df['user_id']==user].drop('user_id',axis=1).values.tolist()

            all_s.append([ Average(x) for x in zip(*s_df) ])


        t2= time.time()
        print(f'finshed averaging users feed in {(t2-t1)/3600} hours')

        new_cols = [ f'user_{x}_mean' for x in ['politeness', 'polarity','subjectivity']]

        tweet_df[new_cols]  = all_s

        tweet_df.reset_index(drop=False)
        tweet_df.set_index('tweet_id', inplace = True)
      
        save_path = f'tweets/{self.hashtag}/{self.hashtag}_USER_POLITE_scores.csv'

        tweet_df.to_csv(save_path)

## RUN

In [7]:
# hashtags = ['avengers','blm','borisjohnson','brexit','climatechange','covid','gaza','loveisland','monkeypox','nhs','olivianewtonjohn','supercup','UkraineWar']

hashtags = ['avengers','blm','monkeypox']

# hashtags = ['avengers']

for hashtag in hashtags:

    a = Analyzer(hashtag)

    a.tweet_analysis()

    print(f'----------------\nDONE \n{hashtag}hashtag  \n----------------')

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB
loaded informer data
loaded in ms tweets
pulled data on 1508 users



 raw feed len 147924



 feed len after drop duplibats 147924



 feed len after not considering the feeds we didnt pull 142143
loaded in user feeds
loaded in necessary data
have 2258 tweets loaded in
now considering 2258 tweets
loaded politeness


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

...

politeness per tweet


All model checkpoint weights were used when initializing RobertaForSequenceClassification.

All the weights of RobertaForSequenceClassification were initialized from the model checkpoint at politeness/results/checkpoint-52500/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RobertaForSequenceClassification for predictions without further training.


NOW PREDICTING POLITENESS SCORES OF USER FEEDS
----


No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Prediction *****
  Num examples = 252296
  Batch size = 8


got prediction scores in 0.12022603578037686 hours
now averaging users feeds
finshed averaging users feed in 0.012336181269751655 hours
----------------
DONE 
monkeypoxhashtag  
----------------




```
# This is formatted as code
```

## EMPTY ROWS
